In [4]:
import cv2

In [3]:
video_cap=cv2.VideoCapture(0)
while True:
    ret,video_data=video_cap.read()
    cv2.imshow("video_live",video_data)
    if cv2.waitKey(10) == ord("a"):
        break
video_cap.release()




In [7]:
import cv2

face_cap = cv2.CascadeClassifier("D:/Python/Python programs/.venv/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")

video_cap = cv2.VideoCapture(0)

while True:
    ret, video_data = video_cap.read()
    if not ret:
        break  

    
    col = cv2.cvtColor(video_data, cv2.COLOR_BGR2GRAY)

    faces = face_cap.detectMultiScale(
        col,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    
    for (x, y, w, h) in faces:
        cv2.rectangle(video_data, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("video_live", video_data)

    if cv2.waitKey(10) == ord("a"):
        break


video_cap.release()
cv2.destroyAllWindows()

In [10]:
#final product
import cv2
from deepface import DeepFace

face_cap = cv2.CascadeClassifier(
    "D:/Python/Python programs/.venv/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml"
)

video_cap = cv2.VideoCapture(0)
frame_count = 0
analysis_interval = 10  


last_results = {}

while True:
    ret, video_data = video_cap.read()
    if not ret:
        break

    frame_count += 1
    gray_frame = cv2.cvtColor(video_data, cv2.COLOR_BGR2GRAY)
    faces = face_cap.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for i, (x, y, w, h) in enumerate(faces):
        face_img = video_data[y:y+h, x:x+w]
        result_key = f"face_{i}"

        if frame_count % analysis_interval == 0:
            try:
                face_img_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
                face_img_rgb = cv2.resize(face_img_rgb, (224, 224))

                result = DeepFace.analyze(
                    face_img_rgb,
                    actions=['age', 'gender', 'emotion'],
                    enforce_detection=False,
                    detector_backend='opencv'
                )[0]

                
                last_results[result_key] = {
                    "gender": result['gender'],
                    "age": int(result['age']),
                    "emotion": result['dominant_emotion']
                }
            except Exception as e:
                print("DeepFace error:", e)
                last_results[result_key] = {
                    "gender": "Unknown",
                    "age": "-",
                    "emotion": "Error"
                }

        
        cv2.rectangle(video_data, (x, y), (x + w, y + h), (0, 255, 0), 2)

        
        info = last_results.get(result_key, {"gender": "", "age": "", "emotion": ""})

        
        cv2.putText(video_data, f"{info['gender']}", (x, y - 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(video_data, f"Age: {info['age']}", (x, y - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(video_data, f"Emotion: {info['emotion']}", (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


        moving_x = x + (frame_count * 5 % max(w - 150, 1))  
        cv2.putText(video_data, "Predator AI Vision", (moving_x, y + h + 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("video_live", video_data)

    if cv2.waitKey(10) == ord("a"):
        break

video_cap.release()
cv2.destroyAllWindows()


Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  5.57it/s]
